Continuando con la lectura de archivos de datos estructurados, a veces no es posible (o es mucho trabajo) hacer el _parsing_ de los datos. Veamos un ejemplo en que esto ocurre:

In [ ]:
let readFile(fileName: string) =  
    let lines = File.ReadAllLines(fileName)
    lines 
    

En este caso, tomamos datos de un archivo con canciones de los Beatles

In [ ]:
let beatlesFile = "../data/The Beatles songs dataset.csv"

let songs = readFile(beatlesFile)

In [ ]:
songs.GetType()

In [ ]:
printfn "%A" songs[0]
printfn "%A" songs[1]

In [ ]:
songs[0..20]
|> Seq.iteri  (fun i s->  printfn $"{i}: {s}")

In [ ]:
let song =   songs[2].Split('"')
printfn "%A" song

In [ ]:
song.Length

Uno puede consultar otros _parsers_, que los hay por doquier, por ejemplo [en esta página](https://www.joelverhagen.com/blog/2020/12/fastest-net-csv-parsers), pero, gracias al tipo de dato estático, existe una módulo que nos puede resolver el problema, a través de _type providers_.

### Type Providers

Un _type provider_  es una biblioteca que nos permite lidiar con tipos particulares datos:

- [CSV type provider](http://fsprojects.github.io/FSharp.Data/library/CsvProvider.html).
- [Html type provider](https://fsprojects.github.io/FSharp.Data/library/HtmlProvider.html).
- [Json type provider](https://fsprojects.github.io/FSharp.Data/library/JsonProvider.html).

son algunos ejemplos. 

La biblioteca `FSharp.Data` es la que usaremos para aprender a leer estos tipos de datos. En un notebook se importa de la siguiente manera:

In [ ]:
#r "nuget: FSharp.Data"

open FSharp.Data

Un _type provider_ genera un tipo de dato a partir de la información que lee desde un archivo. Esto ocurre en el momento de la compilación. Al momento de ejecutar el código, el tipo que se creó puede utilizarse para procesar los datos

In [ ]:
type SongsTypeProvider = FSharp.Data.CsvProvider<"../data/The Beatles songs dataset.csv", HasHeaders=true>

El compilador (y la biblioteca `FSharp.Data`) construyen el _type provider_ utilizando el archivo "../data/The Beatles songs dataset.csv" como plantilla, descubriendo la estructura de los datos. 

Se puede obtener los datos propiamente dichos con:

In [ ]:
let songs = SongsTypeProvider.GetSample()

De este modo, usamos el mismo archivo para crear el tipo y para obtener los datos. Sin embargo, se podría usar dos archivos diferentes, uno como plantilla y otro con los datos. En ese caso, llamamos al método  `.Load`: 

```fsharp
type SongsTypeProvider = FSharp.Data.CsvProvider<"myTemplateDataFile.csv", HasHeaders=true>
let songs = SongsTypeProvider.Load("myRealDataFile.csv")
```


Al crear el tipo de dato, el _type provider_ crea los campos para poder acceder a la información, por ejemplo

In [ ]:
songs.Headers 

nos da los encabezados de cada columna de los datos. Los datos propiamente dichos los encontramos en el campo `.Rows`: 

In [ ]:
songs.Rows
|> Seq.take 20 
|> Seq.iteri  (fun i s ->  printfn $"{i}: {s}")

A partir de los encabezados de las columnas, el _type provider_ construye los campos correspondientes a cada dato:

In [ ]:
songs.Rows
|> Seq.take 20 
|> Seq.iteri  (fun i s ->  printfn $"{i}: {s.Title} by {s.Songwriter}")

In [ ]:
songs.Rows
|> Seq.take 20 
|> Seq.iteri  (fun i s ->  printfn $"{i}: {s.Title} by >{s.``Lead.vocal``}<")

In [ ]:
songs.Rows 
|> Seq.filter (fun r -> r.``Top.50.Billboard``=1)
|> Seq.iter (fun r -> printfn $"Name:{r.Title} position {r.``Top.50.Billboard``}")


#### Eligiendo separadores

Se pueden especificar los separadores al momento de crear el tipo:
```fsharp
CsvProvider<"../data/AirQuality.csv", Separators=";,">
```

### Datos que faltan

El _type provider_ tiene [ciertas reglas para tratar con datos que faltan](https://fsprojects.github.io/FSharp.Data/library/CsvProvider.html#Controlling-the-column-types). Por ejemplo, si el dato que se espera en alguna columna es un número, pero el archivo contiene `NaN`, al crear el dato el _type provider lo reportará como `Double.NaN`. 

Por otro lado, [podemos especificar qué `strings` queremos que se conviertan a `Nan`](https://fsprojects.github.io/FSharp.Data/library/CsvProvider.html#Missing-values):

```fsharp 
CsvProvider<"X,Y,Z\nthis,that,1.0", MissingValues="this,that">
    .GetSample()
    .Rows
```

Además, si preferimos no utilizar las reglas del _type provider_, usamos `PreferOptionals=true` para que genere tipos `option` en el caso 
de datos faltantes:


In [ ]:
type SongsTypeProviderOpt = FSharp.Data.CsvProvider<"../data/The Beatles songs dataset.csv", HasHeaders=true, PreferOptionals=true>

let songsWithOpt = SongsTypeProviderOpt.GetSample()

songsWithOpt.Rows
|> Seq.take 5
|> Seq.iteri  (fun i s ->  printfn $"{i}: {s}")

### Html Provider

También podemos obtener los datos de una página web usando un Html provider:

In [ ]:
[<Literal>]
let url = """https://en.wikipedia.org/wiki/List_of_songs_recorded_by_the_Beatles"""

type WebSongsTypeProvider = FSharp.Data.HtmlProvider<url>


let songs = WebSongsTypeProvider.Load("https://en.wikipedia.org/wiki/List_of_songs_recorded_by_the_Beatles")

In [ ]:
songs.Tables.``Main songsEdit 3``.Rows
|> Seq.map (fun r -> r.Song, r.Year)
|> Seq.filter (fun (s,y) -> y=1968)
|> Seq.iter (fun (s,y) -> printfn $"Name:%s{s} Year:{y}")


### Json Provider

Finalmente, existe un _type provider_  para leer datos en formato JSON (JavaScript Object Notation), que es standard en la transmisión de información en internet.

In [ ]:
[<Literal>]
let tvUrl = "https://raw.githubusercontent.com/mganitombalak/training/master/tv-shows.json"

In [ ]:
type TvListing = JsonProvider<tvUrl>
let tvListing = TvListing.GetSamples()                                   


In [ ]:
tvListing
|> Seq.map (fun t -> (t.Name,t.Rating.Average))
|> Seq.sortByDescending (fun (n,a) -> a)
|> Seq.take 20
|> Seq.iter (fun (n,a) -> printfn $"{n}: {a}")